In [2]:
import numpy as np
import cv2
import os
from random import shuffle
from tqdm import tqdm 

In [3]:
DATA_DIR = "K:\Research\MyMachli\MaskDataset\images"
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'fishCNNMask-{}-{}.model'.format(LR, '2conv-basic')

def createData():
    data = []
    for dir in tqdm(os.listdir(DATA_DIR)):
        img_path = os.path.join(DATA_DIR, dir)
        label = dir.split('_')[-1]
        oneHotLabel = np.zeros(23)
        oneHotLabel[int(label)-1] = 1
        for img in tqdm(os.listdir(img_path)):
            path = os.path.join(img_path, img)
            img = cv2.imread(path)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            data.append([np.array(img), np.array(oneHotLabel)])
        shuffle(data)
        np.save('data.npy',data)
        return data

In [4]:
# data = createData()
data = np.load("data.npy")

In [5]:
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 3], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 23, activation = 'softmax')
convnet = regression(convnet, optimizer = 'adam', learning_rate=LR, loss = 'categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir = 'log')

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [6]:
if(os.path.exists('{}.meta'.format(MODEL_NAME))):
    model.load(MODEL_NAME)
    print('model successfully loaded')

In [7]:
train = data[:-7000]
test = data[6000:]

In [8]:
len(data)

12112

In [9]:
X = np.array([i[0] for i in train])
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test[0:int(len(test)/100)]]) #validation set for x
test_y = [i[1] for i in test[0:int(len(test)/100)]]

print(len(test_x))
print(len(test_y))
print(len(train))
print(len(test))

61
61
5112
6112


In [10]:
print(MODEL_NAME)

fishCNNMask-0.001-2conv-basic.model


In [11]:
model.fit({'input':X}, {'targets': Y}, n_epoch=5, validation_set = ({'input': test_x},
                                                                    {'targets': test_y}),
                                                                      snapshot_step=500, 
                                                                      show_metric = True,
                                                                      run_id = MODEL_NAME)

Training Step: 399  | total loss: 0.00000 | time: 15.022s
| Adam | epoch: 005 | loss: 0.00000 - acc: 1.0000 -- iter: 5056/5112
Training Step: 400  | total loss: 0.00000 | time: 16.217s
| Adam | epoch: 005 | loss: 0.00000 - acc: 1.0000 | val_loss: 0.00000 - val_acc: 1.0000 -- iter: 5112/5112
--
